In [1]:
import pandas as pd
import time
import numpy as np
import re
import pickle
import string, unicodedata
from unidecode import unidecode

## Load Data

In [2]:
#data = pd.read_excel('data/data_komentar_clean_1.xlsx', sheet_name = 'Sheet1')
data = pd.read_excel('data/data_komentar_clean_1.xlsx', sheet_name = 'imbang')
komentar = data['komentar'].tolist()
label = data['label'].tolist()
kode = data['kode'].tolist()

In [3]:
pd.value_counts(data['label'])

bukan spam    1750
spam          1750
Name: label, dtype: int64

## Proses TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
      
tfidf = TfidfVectorizer() #membuat model tf-idf
##Membuat Matrix TF-IDF
tfidf_matrix = tfidf.fit_transform(komentar) #melakukan fit data -> seluruh data komentar
print(tfidf_matrix.shape)
X = tfidf_matrix.toarray() #merubah tipe data ke array
#X= tfidf_matrix
Y = np.array(kode) #labet tiap (komentat [spam dan bukan spam])/ 1:0
y = Y

(3500, 12467)


## Proses Pemodelan

In [5]:
#Import Library
import numpy as np
from sklearn import svm
from sklearn.naive_bayes import ComplementNB

import pylab as pl
from sklearn.svm import SVC
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import GridSearchCV

# Mencari parameter terbaik

## Proposes percobaan untuk masing masing parameter SVM

In [6]:
K = 10 #pembagian data uji dan data latih
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=K) 
C_range = np.arange(1,2.1,0.1)
gamma_range = np.arange(0.1,1.1, 0.1)

C_range_ = []
gamma_range_ = []
rerata_akurasi_svm = []
count=0
for i in range(len(C_range)):
    for j in range(len(gamma_range)):
        C = C_range[i]
        gamma = gamma_range[j]
        akurasi_svm = []
        cSVM = svm.SVC(kernel='rbf', gamma=gamma, C=C) #model
        for train_index, test_index in kf.split(X):
            #membagi data latih dan data uji
            X_train, X_test = X[train_index], X[test_index] #pembagian data latih
            y_train, y_test = y[train_index], y[test_index]
    
            #Melatih Model 
            cSVM = cSVM.fit(X_train, y_train) #SVM
            
            #proses prediksi
            P_SVM = cSVM.predict(X_test)
            akurasi_svm.append(accuracy_score(y_test, P_SVM))
        print("iterasi ke-", count)
        C_range_.append(C)
        gamma_range_.append(gamma)
        
        rr = sum(akurasi_svm)/len(akurasi_svm)
        rerata_akurasi_svm.append(rr)
        count+=1
        #Menampilkan hasil 
        print("-"*30)
        print("C     : ", C)
        print("gamma : ", gamma)
        print("-"*20)
        print("Rata-rata Akurasi SVM: %0.2f" % (sum(akurasi_svm)/len(akurasi_svm)))
        print("="*30)
    
#proses mencari parameter dan akurasi terbaik
x = max(rerata_akurasi_svm)
#al = list(alpha_range)
index = rerata_akurasi_svm.index(x)
round(gamma_range_[index],2)
print("---------------------------")
print("gamma terbaik adalah :", round(gamma_range_[index],2))
print("    C terbaik adalah :", round(C_range_[index],2))
print("             Akurasi :", round(x, 2))


#MENYIMPAN HASIL
dict_komentar  = {"gamma":gamma_range_,"c":C_range_, "akurasi":rerata_akurasi_svm}
df = pd.DataFrame(dict_komentar,columns = ["gamma", 'c', "akurasi"])
writer = pd.ExcelWriter('data/parameter-akurasi-svm_balance.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='svm-parameter-svm')
#df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

iterasi ke- 0
------------------------------
C     :  1.0
gamma :  0.1
--------------------
Rata-rata Akurasi SVM: 0.88
iterasi ke- 1
------------------------------
C     :  1.0
gamma :  0.2
--------------------
Rata-rata Akurasi SVM: 0.90
iterasi ke- 2
------------------------------
C     :  1.0
gamma :  0.30000000000000004
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi ke- 3
------------------------------
C     :  1.0
gamma :  0.4
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi ke- 4
------------------------------
C     :  1.0
gamma :  0.5
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi ke- 5
------------------------------
C     :  1.0
gamma :  0.6
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi ke- 6
------------------------------
C     :  1.0
gamma :  0.7000000000000001
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi ke- 7
------------------------------
C     :  1.0
gamma :  0.8
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi k

iterasi ke- 50
------------------------------
C     :  1.5000000000000004
gamma :  0.1
--------------------
Rata-rata Akurasi SVM: 0.90
iterasi ke- 51
------------------------------
C     :  1.5000000000000004
gamma :  0.2
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi ke- 52
------------------------------
C     :  1.5000000000000004
gamma :  0.30000000000000004
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi ke- 53
------------------------------
C     :  1.5000000000000004
gamma :  0.4
--------------------
Rata-rata Akurasi SVM: 0.92
iterasi ke- 54
------------------------------
C     :  1.5000000000000004
gamma :  0.5
--------------------
Rata-rata Akurasi SVM: 0.92
iterasi ke- 55
------------------------------
C     :  1.5000000000000004
gamma :  0.6
--------------------
Rata-rata Akurasi SVM: 0.92
iterasi ke- 56
------------------------------
C     :  1.5000000000000004
gamma :  0.7000000000000001
--------------------
Rata-rata Akurasi SVM: 0.92
iterasi ke- 57
--

iterasi ke- 99
------------------------------
C     :  1.9000000000000008
gamma :  1.0
--------------------
Rata-rata Akurasi SVM: 0.92
iterasi ke- 100
------------------------------
C     :  2.000000000000001
gamma :  0.1
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi ke- 101
------------------------------
C     :  2.000000000000001
gamma :  0.2
--------------------
Rata-rata Akurasi SVM: 0.91
iterasi ke- 102
------------------------------
C     :  2.000000000000001
gamma :  0.30000000000000004
--------------------
Rata-rata Akurasi SVM: 0.92
iterasi ke- 103
------------------------------
C     :  2.000000000000001
gamma :  0.4
--------------------
Rata-rata Akurasi SVM: 0.92
iterasi ke- 104
------------------------------
C     :  2.000000000000001
gamma :  0.5
--------------------
Rata-rata Akurasi SVM: 0.92
iterasi ke- 105
------------------------------
C     :  2.000000000000001
gamma :  0.6
--------------------
Rata-rata Akurasi SVM: 0.92
iterasi ke- 106
----------------

## Menyimpan hasil percobaan SVM

In [62]:
#MENYIMPAN HASIL
dict_komentar  = {"gamma":gamma_range_,"c":C_range_, "akurasi":rerata_akurasi_svm}
df = pd.DataFrame(dict_komentar,columns = ["gamma", 'c', "akurasi"])
writer = pd.ExcelWriter('data/parameter-akurasi-svm.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='svm-parameter-svm')
#df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

# Mencari Parameter Alpha

In [7]:
K = 10 #pembagian data uji dan data latih
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score#, precision_score

kf = KFold(n_splits=K)

alpha_range = np.arange(0.1,2.1,0.01)

rerata_akurasi_cnb = []

count=0
for i in range(len(alpha_range)): #melakukan interasi sebanyak data parameter C
    alpha = alpha_range[i]
    akurasi_cnb = []
    comNB = ComplementNB(alpha = alpha) #model
    tic = time.perf_counter()
    for train_index, test_index in kf.split(X):
            #membagi data latih dan data uji
        X_train, X_test = X[train_index], X[test_index] #pembagian data latih
        y_train, y_test = y[train_index], y[test_index]
    
        #Melatih Model 
        mCNB = comNB.fit(X_train, y_train) 
            
        #proses prediksi
        P_CNB = mCNB.predict(X_test)
        akurasi_cnb.append(accuracy_score(y_test, P_CNB))
    toc = time.perf_counter()
    print("iterasi ke-", count)    
    rr = sum(akurasi_cnb)/len(akurasi_cnb)
    rerata_akurasi_cnb.append(rr)
    count+=1
    #Menampilkan hasil akhir
    print("---------------------------------")
    print("alpha : ", alpha)
    print("----------------------------")
    print("Rata-rata Akurasi cnb: %0.2f" % (sum(akurasi_cnb)/len(akurasi_cnb)))
    print("---------------------------------", toc-tic, "second")
    
#proses mencari parameter dan akurasi terbaik
x = max(rerata_akurasi_cnb)
index = rerata_akurasi_cnb.index(x)
print("---------------------------")
print("alpha terbaik adalah :", round(alpha_range[index],2))
print("             Akurasi :", round(x, 2))

round_akurasi_cnb = []
for i in rerata_akurasi_cnb:
    round_akurasi_cnb.append(round(i,2))
dict_komentar  = {"alpha":alpha_range,"akurasi":rerata_akurasi_cnb, "Akurasi round":round_akurasi_cnb}
df = pd.DataFrame(dict_komentar,columns = ["alpha","akurasi","Akurasi round"])
writer = pd.ExcelWriter('data/parameter-akurasi-cnb_balance2.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='cnb-parameter-cnb')

#menyimpan file
writer.save()

iterasi ke- 0
---------------------------------
alpha :  0.1
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.9045582 second
iterasi ke- 1
---------------------------------
alpha :  0.11
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.2821032 second
iterasi ke- 2
---------------------------------
alpha :  0.12
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.275790100000002 second
iterasi ke- 3
---------------------------------
alpha :  0.13
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.262827900000001 second
iterasi ke- 4
---------------------------------
alpha :  0.13999999999999999
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.2774237999999976 second
iterasi ke- 5
---------------------------------
alpha :  0.14999999999999997
----------------------------
Rata-

iterasi ke- 43
---------------------------------
alpha :  0.5299999999999998
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.203342600000013 second
iterasi ke- 44
---------------------------------
alpha :  0.5399999999999998
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.250226499999997 second
iterasi ke- 45
---------------------------------
alpha :  0.5499999999999998
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.221705500000013 second
iterasi ke- 46
---------------------------------
alpha :  0.5599999999999997
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.263683100000009 second
iterasi ke- 47
---------------------------------
alpha :  0.5699999999999997
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.254779600000006 second
iterasi ke- 48
--------------------

iterasi ke- 86
---------------------------------
alpha :  0.9599999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.274461099999996 second
iterasi ke- 87
---------------------------------
alpha :  0.9699999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.267419199999949 second
iterasi ke- 88
---------------------------------
alpha :  0.9799999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.1978674999999726 second
iterasi ke- 89
---------------------------------
alpha :  0.9899999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.242116899999985 second
iterasi ke- 90
---------------------------------
alpha :  0.9999999999999996
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.238888099999997 second
iterasi ke- 91
-------------------

iterasi ke- 129
---------------------------------
alpha :  1.3899999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.24858590000008 second
iterasi ke- 130
---------------------------------
alpha :  1.3999999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.2284365000000435 second
iterasi ke- 131
---------------------------------
alpha :  1.4099999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.284284800000023 second
iterasi ke- 132
---------------------------------
alpha :  1.4199999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.274561800000015 second
iterasi ke- 133
---------------------------------
alpha :  1.4299999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 5.2514867000001 second
iterasi ke- 134
----------------

iterasi ke- 172
---------------------------------
alpha :  1.8199999999999992
----------------------------
Rata-rata Akurasi cnb: 0.91
--------------------------------- 5.293393400000014 second
iterasi ke- 173
---------------------------------
alpha :  1.8299999999999992
----------------------------
Rata-rata Akurasi cnb: 0.91
--------------------------------- 5.30948920000003 second
iterasi ke- 174
---------------------------------
alpha :  1.8399999999999992
----------------------------
Rata-rata Akurasi cnb: 0.91
--------------------------------- 5.342879499999981 second
iterasi ke- 175
---------------------------------
alpha :  1.8499999999999992
----------------------------
Rata-rata Akurasi cnb: 0.91
--------------------------------- 5.2766132000000425 second
iterasi ke- 176
---------------------------------
alpha :  1.8599999999999992
----------------------------
Rata-rata Akurasi cnb: 0.91
--------------------------------- 5.276418700000022 second
iterasi ke- 177
--------------

## Menyimpan hasil percobaan CNB

In [10]:
round_akurasi_cnb = []
for i in rerata_akurasi_cnb:
    round_akurasi_cnb.append(round(i,2))
dict_komentar  = {"alpha":alpha_range,"akurasi":rerata_akurasi_cnb, "Akurasi round":round_akurasi_cnb}
df = pd.DataFrame(dict_komentar,columns = ["alpha","akurasi","Akurasi round"])
writer = pd.ExcelWriter('data/parameter-akurasi-cnb_balance.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='cnb-parameter-cnb')

#menyimpan file
writer.save()